In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select

import pandas as pd
from tabulate import tabulate
import os
import requests
import datetime

import shutil
import time


In [2]:
def page_is_loading(driver):
    while True:
        x = driver.execute_script("return document.readyState")
        if x == "complete":
            return True
        else:
            return False

In [3]:
def apply_soup(dominio):
    url = dominio
    soupDriver = None
    driver = webdriver.Chrome()
    #driver.implicitly_wait(5)    
    driver.get(url)
    time.sleep(1)     
    #driver.maximize_window();    
    maximo = driver.execute_script("return document.body.scrollHeight")
    parcial = 400
    while parcial < maximo:
        posicion = "window.scrollTo(0,"+str(parcial)+")"
        driver.execute_script(posicion)        
        while not page_is_loading(driver):
            continue

        maximo = driver.execute_script("return document.body.scrollHeight")
        parcial = parcial + 650 
                            
    return driver

## Obtener las cedulas a procesar

In [4]:
# lEER DATOS DESDE EXCEL
ipath = 'Cedulas a consultar CO.xlsx'
dfCedulas = pd.read_excel(ipath)
dfCedulas.head()


,Cedula
0,23251792
1,23253303
2,23433016
3,4414169
4,9859830


In [5]:
for indice_fila, fila in dfCedulas.iterrows():
    print(str(indice_fila) + " - " + str(fila['Cedula']))

0 - 23251792
1 - 23253303
2 - 23433016
3 - 4414169
4 - 9859830
5 - 1054769856
6 - 1054857825
7 - 1055750262
8 - 1058819793
9 - 1058845841
10 - 1060651149
11 - 1060652235
12 - 1060654474
13 - 1060654586
14 - 1070919081
15 - 1084037531
16 - 1085308364
17 - 1087049041
18 - 1088332181
19 - 1089720081
20 - 1091681318
21 - 1093229202
22 - 1109294164
23 - 1110489453
24 - 1110529183
25 - 1112790777
26 - 1115086181
27 - 1124860764
28 - 1127772276
29 - 1136889150
30 - 1152693247
31 - 1193103447
32 - 1193412321
33 - 1193524007
34 - 1225089344


# Obtener el estado de las cedulas

In [6]:
xDate = datetime.datetime.now()
indice = 0 #counter 
datalist = []
## representacion de fecha y hora
inicio = datetime.datetime.now()
ahora = time.strftime("%c")
print("Fecha y hora Inicio : " + time.strftime("%c"))
dominio = 'https://defunciones.registraduria.gov.co/'
soupDriver = apply_soup(dominio)

for indice_fila, fila in dfCedulas.iterrows():
    result={}
    cedula = fila['Cedula']      
    soupDriver.find_element(By.NAME, "nuip").send_keys(str(cedula) + Keys.RETURN)
    time.sleep(1)
    fecha = soupDriver.find_element(By.XPATH, '/html/body/app-root/section/div[2]/div/div/div/div/h5').text
    cedulaResp = soupDriver.find_element(By.XPATH, '/html/body/app-root/section/div[2]/div/div/div/div/p/span[1]').text
    
    try:
        estado = soupDriver.find_element(By.XPATH, '/html/body/app-root/section/div[2]/div/div/div/div/p/span[2]').text
    except Exception as e:
        estado = "no existe en el archivo nacional de identificación"
    fecha = fecha.replace("Fecha Consulta: ","")
    result["fecha_consulta"] = fecha
    result["cedula_Resp"] = cedulaResp
    result["estado"] = estado
    datalist.append(result)
    print(str(indice) + " - " + result["fecha_consulta"] + " - " + result["cedula_Resp"] + " - " + result["estado"] )
    indice += 1 

soupDriver.quit()
df=pd.DataFrame(datalist)

formato = "%d-%m-%Y"    
writer = pd.ExcelWriter('Registraduría Nacional_CO'+'_'+xDate.strftime(formato) + '.xlsx')
df.to_excel(writer,'Resultado')
writer.close()
    
fin = datetime.datetime.now()
print ("El tiempo transcurrido fue de: ", fin - inicio)


Fecha y hora Inicio : Tue May 14 14:31:43 2024
0 - 14/05/2024 - 23251792 - Cancelada por Muerte
1 - 14/05/2024 - 23253303 - Vigente (Vivo)
2 - 14/05/2024 - 23433016 - Vigente (Vivo)
3 - 14/05/2024 - 4414169 - Vigente (Vivo)
4 - 14/05/2024 - 9859830 - Vigente (Vivo)
5 - 14/05/2024 - 1054769856 - no existe en el archivo nacional de identificación
6 - 14/05/2024 - 1054857825 - Vigente (Vivo)
7 - 14/05/2024 - 1055750262 - Vigente (Vivo)
8 - 14/05/2024 - 1058819793 - Vigente (Vivo)
9 - 14/05/2024 - 1058845841 - Vigente (Vivo)
10 - 14/05/2024 - 1060651149 - Vigente (Vivo)
11 - 14/05/2024 - 1060652235 - Vigente (Vivo)
12 - 14/05/2024 - 1060654474 - Vigente (Vivo)
13 - 14/05/2024 - 1060654586 - Vigente (Vivo)
14 - 14/05/2024 - 1070919081 - Vigente (Vivo)
15 - 14/05/2024 - 1084037531 - no existe en el archivo nacional de identificación
16 - 14/05/2024 - 1085308364 - Vigente (Vivo)
17 - 14/05/2024 - 1087049041 - Vigente (Vivo)
18 - 14/05/2024 - 1088332181 - Vigente (Vivo)
19 - 14/05/2024 - 10897